In [2]:
from typing import NamedTuple, List
from functools import reduce
from __future__ import annotations

RAW = """939
7,13,x,x,59,x,31,19"""


class Bus(NamedTuple):
    ts:int
    buses: List[int]

    @staticmethod
    def parse_bus(inputs:str)->Bus:
        lookup = [int(num)
            for row in inputs.split("\n")
            for num in row.split(",")
            if num != "x"]
        return Bus(ts=lookup[0], buses=lookup[1:])
    

def earliest_bus(bus:Bus) -> int:
    start_ts = bus.ts
    while True:
        for b in bus.buses:
            not_arrived = start_ts % b == 0
            #print(b, not_arrived, start_ts)
            if not_arrived:
                return (start_ts - bus.ts) * b
        start_ts += 1

bus = Bus.parse_bus(RAW)
assert earliest_bus(bus) == 295

with open("puzzle_inputs/day13.txt") as f:
    inputs = f.read()
bus = Bus.parse_bus(inputs)
earliest_bus(bus)

Part 2 unresolved

- checked Joel G solution 

- check chinese remainder theorem resolution here https://www.youtube.com/watch?v=zIFehsBHB8o

```
7,13,x,x,59,x,31,19
find t such that
t % 7 == 0
t % 13 == 13 - 1
skip 2
skip 3
t % 59 == 59 - 4
skip 5
t % 31 == 31 - 6
t % 19 == 19 - 7
```

In [10]:
def make_factors(inputs:str)-> List[List[int],List[int]]:
    raw_buses = [int(x) if x != "x" else x
             for x in inputs.split("\n")[1:][0].split(",")]
    factors = range(len(raw_buses))
    bus_factors = [(b, (b - f) % b)
              for b, f in zip(raw_buses, factors)
              if b != 'x']
    
    bus = [b for b, n in bus_factors]
    factors = [n for b, n in bus_factors]
    return [bus, factors]
    

# chinese remainder code from
# https://rosettacode.org/wiki/Chinese_remainder_theorem#Python_3.6

# n are the buses and a the factors ( bus - t) % bus
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod


def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

bus, factors = make_factors(inputs=RAW)
assert chinese_remainder(bus, factors) == 1068781

bus, factors = make_factors(inputs=inputs)

chinese_remainder(bus, factors)